# Oversampling the minority class

In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
df = pd.read_excel(r'C:/BEPEC Python Material/Machine Learning/Logistic Regression/credit_card.xlsx', engine='openpyxl')
df.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,0,0,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,0,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0


In [3]:
column_names = df.loc[0,:]
column_names = column_names.to_list()
l1 = df.columns
l1 = l1.to_list()
df.columns = column_names
df.rename(columns= {'PAY_0':'PAY_1'}, inplace=True)
df.drop([0], inplace=True)
column_names = df.columns
for column in column_names:
    df[column] = pd.to_numeric(df[column]) 
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,0,0,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,0,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,5,50000,1,2,1,57,0,0,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [4]:
df.drop(columns={'ID'}, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 1 to 30000
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   LIMIT_BAL                   30000 non-null  int64
 1   SEX                         30000 non-null  int64
 2   EDUCATION                   30000 non-null  int64
 3   MARRIAGE                    30000 non-null  int64
 4   AGE                         30000 non-null  int64
 5   PAY_1                       30000 non-null  int64
 6   PAY_2                       30000 non-null  int64
 7   PAY_3                       30000 non-null  int64
 8   PAY_4                       30000 non-null  int64
 9   PAY_5                       30000 non-null  int64
 10  PAY_6                       30000 non-null  int64
 11  BILL_AMT1                   30000 non-null  int64
 12  BILL_AMT2                   30000 non-null  int64
 13  BILL_AMT3                   30000 non-null  int64
 14  BILL_A

# Fourth method is winner

In [6]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(24000, 23)
(24000,)
(6000, 23)
(6000,)


In [9]:
from sklearn import neighbors

model = neighbors.KNeighborsClassifier()
model.fit(X_train,y_train)
test_predicted = model.predict(X_test)
train_predicted = model.predict(X_train)


In [11]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test, test_predicted),"\n------------------")
print(metrics.confusion_matrix(y_train, train_predicted))

[[4326  377]
 [1046  251]] 
------------------
[[17785   876]
 [ 3547  1792]]


In [17]:
print(metrics.classification_report(y_test, test_predicted  ))

              precision    recall  f1-score   support

           0       0.81      0.92      0.86      4703
           1       0.40      0.19      0.26      1297

    accuracy                           0.76      6000
   macro avg       0.60      0.56      0.56      6000
weighted avg       0.72      0.76      0.73      6000



In [21]:
from sklearn.metrics import f1_score
print(f1_score(y_test , test_predicted , average = 'weighted'))

0.7294959436262665


# First method

In [19]:
from collections import Counter
from imblearn.over_sampling import SMOTE , ADASYN
ovr_sample = SMOTE(sampling_strategy='auto',k_neighbors=7 )

X_over , y_over = ovr_sample.fit_resample(X,y)
print(Counter(y_over))

Counter({1: 23364, 0: 23364})


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_over,y_over, test_size=0.2, random_state = 0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(37382, 23)
(37382,)
(9346, 23)
(9346,)


In [21]:
from sklearn import neighbors

model = neighbors.KNeighborsClassifier()
model.fit(X_train,y_train)
test_predicted = model.predict(X_test)
train_predicted = model.predict(X_train)


In [22]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test, test_predicted),"\n------------------")
print(metrics.confusion_matrix(y_train, train_predicted))

[[2897 1716]
 [ 657 4076]] 
------------------
[[13683  5068]
 [ 1237 17394]]


In [23]:
print(metrics.classification_report(y_test, test_predicted))

              precision    recall  f1-score   support

           0       0.82      0.63      0.71      4613
           1       0.70      0.86      0.77      4733

    accuracy                           0.75      9346
   macro avg       0.76      0.74      0.74      9346
weighted avg       0.76      0.75      0.74      9346



# Second Method

In [51]:

X_resampled, y_resampled = ADASYN().fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 23364), (1, 23948)]


In [52]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size=0.2, random_state = 0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(37849, 23)
(37849,)
(9463, 23)
(9463,)


In [71]:
model = neighbors.KNeighborsClassifier(n_neighbors=1)
model.fit(X_train,y_train)
test_predicted = model.predict(X_test)
train_predicted = model.predict(X_train)


In [72]:
print(metrics.confusion_matrix(y_test, test_predicted),"\n------------------")
print(metrics.confusion_matrix(y_train, train_predicted))

[[3241 1406]
 [ 305 4511]] 
------------------
[[18700    17]
 [   19 19113]]


In [73]:
print(metrics.classification_report(y_test, test_predicted))

              precision    recall  f1-score   support

           0       0.91      0.70      0.79      4647
           1       0.76      0.94      0.84      4816

    accuracy                           0.82      9463
   macro avg       0.84      0.82      0.82      9463
weighted avg       0.84      0.82      0.82      9463



# Second method

In [19]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_over, y_over = ros.fit_resample(X, y)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0, 23364), (1, 23364)]


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_over,y_over, test_size=0.2, random_state = 0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(37382, 23)
(37382,)
(9346, 23)
(9346,)


In [21]:
model = neighbors.KNeighborsClassifier()
model.fit(X_train,y_train)
test_predicted = model.predict(X_test)
train_predicted = model.predict(X_train)


In [22]:
print(metrics.confusion_matrix(y_test, test_predicted),"\n------------------")
print(metrics.confusion_matrix(y_train, train_predicted))

[[2844 1769]
 [ 898 3835]] 
------------------
[[13506  5245]
 [ 1608 17023]]


In [23]:
print(metrics.classification_report(y_test, test_predicted))

              precision    recall  f1-score   support

           0       0.76      0.62      0.68      4613
           1       0.68      0.81      0.74      4733

    accuracy                           0.71      9346
   macro avg       0.72      0.71      0.71      9346
weighted avg       0.72      0.71      0.71      9346



# Third Method

In [34]:
df.head(2)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,2,2,0,0,0,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,0,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1


In [46]:
from imblearn.over_sampling import SMOTENC
sm = SMOTENC(random_state=42, categorical_features=[1,2,3], k_neighbors=9)
X_res, y_res = sm.fit_resample(X, y)


In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res, test_size=0.2, random_state = 0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(37382, 23)
(37382,)
(9346, 23)
(9346,)


In [48]:
model = neighbors.KNeighborsClassifier()
model.fit(X_train,y_train)
test_predicted = model.predict(X_test)
train_predicted = model.predict(X_train)


In [49]:
print(metrics.confusion_matrix(y_test, test_predicted),"\n------------------")
print(metrics.confusion_matrix(y_train, train_predicted))

[[2870 1743]
 [ 780 3953]] 
------------------
[[13823  4928]
 [ 1527 17104]]


In [50]:
print(metrics.classification_report(y_test, test_predicted))

              precision    recall  f1-score   support

           0       0.79      0.62      0.69      4613
           1       0.69      0.84      0.76      4733

    accuracy                           0.73      9346
   macro avg       0.74      0.73      0.73      9346
weighted avg       0.74      0.73      0.73      9346



# Fourth method

In [96]:
from imblearn.over_sampling import KMeansSMOTE
sm = KMeansSMOTE(random_state=42, cluster_balance_threshold=0.01)
X_res, y_res = sm.fit_resample(X, y)

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res, test_size=0.2, random_state = 0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(37388, 23)
(37388,)
(9347, 23)
(9347,)


In [98]:
model = neighbors.KNeighborsClassifier()
model.fit(X_train,y_train)
test_predicted = model.predict(X_test)
train_predicted = model.predict(X_train)


In [99]:
print(metrics.confusion_matrix(y_test, test_predicted),"\n------------------")
print(metrics.confusion_matrix(y_train, train_predicted))

[[3522 1052]
 [ 689 4084]] 
------------------
[[15606  3184]
 [ 2141 16457]]


In [100]:
print(metrics.classification_report(y_test, test_predicted))

              precision    recall  f1-score   support

           0       0.84      0.77      0.80      4574
           1       0.80      0.86      0.82      4773

    accuracy                           0.81      9347
   macro avg       0.82      0.81      0.81      9347
weighted avg       0.82      0.81      0.81      9347



# Always perform resampling on train data ( first split data ) to avoid Data leakage

In [22]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(24000, 23)
(24000,)
(6000, 23)
(6000,)


In [25]:
X_train.head(2)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
3226,20000,2,3,1,44,0,0,2,0,0,...,17980,18780,0,0,3000,0,1000,1000,0,0
11816,260000,2,2,2,30,0,0,0,0,0,...,274,165,333,165,165,274,165,333,165,293


In [27]:
training_set = pd.concat([X_train , y_train], axis = 1)
training_set.head(2)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
3226,20000,2,3,1,44,0,0,2,0,0,...,18780,0,0,3000,0,1000,1000,0,0,0
11816,260000,2,2,2,30,0,0,0,0,0,...,165,333,165,165,274,165,333,165,293,0


In [31]:
defaulters = training_set[training_set['default payment next month'] ==  1]
non_defaulters = training_set[training_set['default payment next month'] ==  0]
print(defaulters.shape)
print(non_defaulters.shape)
X_train.shape

(5339, 24)
(18661, 24)


(24000, 23)

In [46]:
# Undersampling the majority class

from sklearn.utils import resample

undersample = resample(non_defaulters, 
                       replace=True, 
                       n_samples=len(defaulters), #set the number of samples to equal the number of the minority class
                       random_state=42)
undersampled_train = pd.concat([undersample , defaulters])
undersampled_train['default payment next month'].value_counts(normalize = True)


1    0.5
0    0.5
Name: default payment next month, dtype: float64

In [49]:
undersampled_train_X = undersampled_train.iloc[:,:-1]
undersampled_train_y = undersampled_train.iloc[:,-1]
print(undersampled_train_X.shape)
print(undersampled_train_y.shape)

model = neighbors.KNeighborsClassifier()
model.fit(undersampled_train_X,undersampled_train_y)
test_predicted = model.predict(X_test)
train_predicted = model.predict(undersampled_train_X)


print(metrics.confusion_matrix(y_test, test_predicted),"\n------------------")
print(metrics.confusion_matrix(undersampled_train_y, train_predicted))

print(metrics.classification_report(y_test, test_predicted  ))

(10678, 23)
(10678,)
[[2653 2050]
 [ 511  786]] 
------------------
[[3977 1362]
 [1367 3972]]
              precision    recall  f1-score   support

           0       0.84      0.56      0.67      4703
           1       0.28      0.61      0.38      1297

    accuracy                           0.57      6000
   macro avg       0.56      0.59      0.53      6000
weighted avg       0.72      0.57      0.61      6000

